### Imports

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# get the data
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 5s 0us/step


### Building functions

In [3]:
def preprocess_images(images):
    return images.reshape(-1, 784).astype(np.float32) / 255


def preprocess_labels(labels):
    return labels.reshape(-1).astype(np.int32)

### Preprocessing

In [5]:
# Preprocessing train and test data
train_images = preprocess_images(train_images)
test_images = preprocess_images(test_images)
train_labels = preprocess_labels(train_labels)
test_labels = preprocess_labels(test_labels)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(60000).batch(128).repeat()
#test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(128)

### Defining Model

In [27]:
# define the model first, from input to output

# this is a super deep model, cool!
n_units = 100
n_layers = 8
w_range = 0.4

In [ ]:
# just set up a "chain" of hidden layers
# model is represented by a list where each element is a layer,
# and each layer is in turn a list of the layer variables (w, b)

# first layer goes from n_input to n_hidden
w_input = tf.Variable(tf.random.uniform([784, n_units], -w_range, w_range),
                      name="w0")
b_input = tf.Variable(tf.zeros(n_units), name="b0")
layers = [[w_input, b_input]]

In [ ]:
# all other hidden layers go from n_hidden to n_hidden
for layer in range(n_layers - 1):
    w = tf.Variable(tf.random.uniform([n_units, n_units], -w_range, w_range),
                    name="w" + str(layer+1))
    b = tf.Variable(tf.zeros(n_units), name="b" + str(layer+1))
    layers.append([w, b])

In [ ]:
# finally add the output layer
w_out = tf.Variable(tf.random.uniform([n_units, 10], -w_range, w_range),
                    name="wout")
b_out = tf.Variable(tf.zeros(10), name="bout")
layers.append([w_out, b_out])

# flatten the layers to get a list of variables
all_variables = [variable for layer in layers for variable in layer]

In [ ]:
def model_forward(inputs):
    x = inputs
    for w, b in layers[:-1]:
        x = tf.nn.sigmoid(tf.matmul(x, w) + b) 
        # problem dignoised with using sigmoid rather than using relu. As sigmoid returns value between the range of 0to1. 
        # Formula of Cross-Entropy Loss Function is ti log(pi) and when the value of softmax is 0 then log return NAN value.
        # Before diagnosing relu activation function was applies which retuns the max(0, 1), which means it will return 0
        # if there will be negative value 
        
    logits = tf.matmul(x, layers[-1][0]) + layers[-1][1]

    return logits

In [38]:
lr = 0.1
train_steps = 2000
for step, (img_batch, lbl_batch) in enumerate(train_data):
    if step > train_steps:
        break

    with tf.GradientTape() as tape:
        # here we just run all the layers in sequence via a for-loop
        logits = model_forward(img_batch)
        
        #print(logits) 
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=lbl_batch))

    grads = tape.gradient(xent, all_variables)
    for grad, var in zip(grads, all_variables):
        var.assign_sub(lr*grad)

    if not step % 100:
        preds = tf.argmax(logits, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch), tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))

Loss: 2.2973685264587402 Accuracy: 0.140625
Loss: 2.3133749961853027 Accuracy: 0.078125
Loss: 2.3098978996276855 Accuracy: 0.09375
Loss: 2.3052783012390137 Accuracy: 0.0859375
Loss: 2.3113157749176025 Accuracy: 0.1328125
Loss: 2.2948718070983887 Accuracy: 0.125
Loss: 2.3119969367980957 Accuracy: 0.09375
Loss: 2.307298183441162 Accuracy: 0.1015625
Loss: 2.3248047828674316 Accuracy: 0.1015625
Loss: 2.285761594772339 Accuracy: 0.140625
Loss: 2.3152990341186523 Accuracy: 0.1015625
Loss: 2.300934314727783 Accuracy: 0.1171875
Loss: 2.300442695617676 Accuracy: 0.09375
Loss: 2.297856330871582 Accuracy: 0.140625
Loss: 2.3062071800231934 Accuracy: 0.09375
Loss: 2.3084402084350586 Accuracy: 0.078125
Loss: 2.3271195888519287 Accuracy: 0.0625
Loss: 2.296616315841675 Accuracy: 0.09375
Loss: 2.3044378757476807 Accuracy: 0.1328125
Loss: 2.303013801574707 Accuracy: 0.109375
Loss: 2.3019509315490723 Accuracy: 0.09375


### visualizing Test Accuracy

In [39]:
test_preds = model_forward(test_images)
test_preds = tf.argmax(test_preds, axis=1, output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, test_labels), tf.float32))
print("Final test accuracy: {}".format(acc))

Final test accuracy: 0.11349999904632568
